<br>
# **자연어와 Deep Learning**
Tensorflow

# **1 LSTM 단어유추모델**

<br>
## **1 데이터 정의**

In [1]:
import tensorflow as tf
import numpy as np
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
num_dic  = {n: i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)

In [2]:
def make_batch(seq_data):
    input_batch, target_batch = [], []
    for seq in seq_data: 
        input_num = [num_dic[n] for n in seq[:-1]]  
        target    = num_dic[seq[-1]]                
        input_batch.append(np.eye(dic_len)[input_num])
        target_batch.append(target)          
    return input_batch, target_batch

<br></br>
## **2 모델의 정의**

In [3]:
tf.reset_default_graph()
learning_rate = 0.01
n_step = 3           
n_hidden, total_epoch = 64, 30
n_input = n_class = dic_len

In [4]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32,   [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class])) 
b = tf.Variable(tf.random_normal([n_class]))

In [5]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)    
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)         
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

In [6]:
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model   = tf.matmul(outputs, W) + b

In [7]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

<br></br>
## **3 모델의 학습**

In [8]:
%%time
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})
    if epoch % 4 == 0:
        print('Epoch: {:.4f} cost = {:.6f}'.format(epoch + 1, loss))
print('최적화 완료!')

Epoch: 1.0000 cost = 3.017860
Epoch: 5.0000 cost = 1.565631
Epoch: 9.0000 cost = 0.826021
Epoch: 13.0000 cost = 0.471006
Epoch: 17.0000 cost = 0.167337
Epoch: 21.0000 cost = 0.112551
Epoch: 25.0000 cost = 0.139093
Epoch: 29.0000 cost = 0.106368
최적화 완료!
CPU times: user 993 ms, sys: 341 ms, total: 1.33 s
Wall time: 1.25 s


<br>
## **4 학습 모델의 평가**

In [9]:
%%time
prediction       = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)               
accuracy         = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val     = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

CPU times: user 79.6 ms, sys: 0 ns, total: 79.6 ms
Wall time: 75 ms


In [10]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:2] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:-2] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)
sess.close()


=== 예측 결과 ===
입력값: ['wo ', 'wo ', 'de ', 'di ', 'co ', 'co ', 'lo ', 'lo ', 'ki ', 'ki ']
예측값: ['wod', 'wod', 'dep', 'die', 'col', 'col', 'lod', 'loe', 'kis', 'kid']
정확도: 0.9


<br>
# **2 Seq2Seq 번역모델**

<br>
## **1 데이터의 정의**

In [11]:
char_arr = [ c for c in 'SPabcdefghijklmnopqrstuvwxyz나놀녀단랑무사소스어이키E'] 
num_dic  = { n : i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)

In [12]:
seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], 
            ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])] 
        target = [num_dic[n] for n in (seq[1] + 'E')]
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
    return input_batch, output_batch, target_batch

<br>
## **2 모델의 정의**

In [13]:
import tensorflow as tf
import numpy as np

In [14]:
tf.reset_default_graph()
learning_rate = 0.01
n_hidden, total_epoch = 128, 100
n_class = n_input = dic_len

enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])  # [batch size, time steps]

In [15]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob = 0.5) 
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,dtype=tf.float32)

In [16]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, 
                                            initial_state = enc_states, 
                                            dtype = tf.float32)

In [17]:
model = tf.layers.dense(outputs, n_class, activation=None)
cost  = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits = model, labels = targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

<br>
## **3 모델의 학습**

In [18]:
%%time
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    if epoch % 9 == 0 :
        print('Epoch: {:4d}  cost = {:.6f}'.format((epoch + 1),loss))
print('최적화 완료!')

Epoch:    1  cost = 3.712980
Epoch:   10  cost = 0.168641
Epoch:   19  cost = 0.062683
Epoch:   28  cost = 0.003721
Epoch:   37  cost = 0.001881
Epoch:   46  cost = 0.000699
Epoch:   55  cost = 0.000727
Epoch:   64  cost = 0.000532
Epoch:   73  cost = 0.001410
Epoch:   82  cost = 0.000505
Epoch:   91  cost = 0.000470
Epoch:  100  cost = 0.000463
최적화 완료!
CPU times: user 989 ms, sys: 104 ms, total: 1.09 s
Wall time: 722 ms


<br>
## **4 모델의 검증**

In [19]:
%%time
def translate(word):
    seq_data = [word, 'P' * len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2)   # [None, None, n_input]
    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})
    decoded = [char_arr[i] for i in result[0]]
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    return translated

print('\n=== 번역 테스트 ===')
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))
sess.close()


=== 번역 테스트 ===
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 단어이
CPU times: user 318 ms, sys: 38 µs, total: 318 ms
Wall time: 298 ms
